In [20]:
import torch

# Load YOLOv5 model (pre-trained on COCO dataset)
model = torch.hub.load('./', 'yolov5s', source='local')  

YOLOv5  v7.0-411-gf4d8a84c Python-3.12.6 torch-2.4.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [21]:
import cv2
import numpy as np
from IPython.display import display, Image
from pathlib import Path

In [22]:
def detect_humans_from_video(video_path, save_output=False, output_path='output.mp4'):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get the width and height of the video frame
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    if save_output:
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Perform inference
        results = model(frame)

        # Filter only 'person' class
        results = results.pandas().xyxy[0]
        persons = results[results['name'] == 'person']

        # Draw bounding boxes for detected persons
        for index, row in persons.iterrows():
            x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
            label = row['name']
            confidence = row['confidence']

            # Draw rectangle
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'{label} {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        if save_output:
            out.write(frame)

        # Display the resulting frame (real-time detection)
        cv2.imshow('YOLOv5 Human Detection', frame)

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    if save_output:
        out.release()
    cv2.destroyAllWindows()


In [23]:
video_path = './854181-hd_1920_1080_25fps.avi'
detect_humans_from_video(video_path, save_output=True, output_path='output.mp4')

Error: Could not open video.
